<a href="https://colab.research.google.com/github/myan105/Hospital-Analysis1/blob/main/Hospital_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()


Saving HOSP DATA (top 20).csv to HOSP DATA (top 20).csv
Saving HOSP DATA vol by year.csv to HOSP DATA vol by year.csv


In [20]:
import pandas as pd
import io


df = pd.read_csv(io.BytesIO(uploaded['HOSP DATA (top 20).csv']))
df_vol_by_year = pd.read_csv(io.BytesIO(uploaded['HOSP DATA vol by year.csv']))
# Check data
df.head(), df_vol_by_year.head()


(          City        Country       County      My Hospital  \
 0     Glendale  United States  Los Angeles  Other Hospitals   
 1    Lancaster  United States  Los Angeles  Other Hospitals   
 2  Los Angeles  United States  Los Angeles  Other Hospitals   
 3  Los Angeles  United States  Los Angeles  Other Hospitals   
 4  Los Angeles  United States  Los Angeles  Other Hospitals   
 
                                     Name       State    ZIP  Avg Age '22  \
 0            Adventist Health - Glendale  California  91206        50.21   
 1               Antelope Valley Hospital  California  93534        37.12   
 2          Cedars Sinai - Medical Center  California  90048        49.37   
 3       Childrens Hospital - Los Angeles  California  90027         6.62   
 4  Hollywood Presbyterian Medical Center  California  90027        45.06   
 
   Avg Costs per Disch  Avg LOS '22  ...  Hosp latitude  Hosp Longitude  \
 0                $23K         5.93  ...          34.16         -118.20   


In [21]:
# Data Cleaning
df = df_top20.dropna()
df = pd.get_dummies(df_top20, drop_first=True)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [25]:
# For Cost per Discharge, create a numeric variable from binary columns
cost_cols = ['Avg Costs per Disch_$20K', 'Avg Costs per Disch_$21K', 'Avg Costs per Disch_$22K',
             'Avg Costs per Disch_$23K', 'Avg Costs per Disch_$24K', 'Avg Costs per Disch_$25K']

# Convert binary columns to numeric values
cost_values = [20000, 21000, 22000, 23000, 24000, 25000]
df['Avg_Cost_Per_Disch'] = df[cost_cols].idxmax(axis=1).str.extract('(\d+)').astype(float)

# Drop binary cost columns after creating numeric column
X = df.drop(cost_cols + ["Avg LOS '22", 'ZIP'], axis=1)

y_los = df["Avg LOS '22"]
y_cost = df["Avg_Cost_Per_Disch"]

# Split dataset properly for each prediction
X_train_los, X_test_los, y_train_los, y_test_los = train_test_split(X, y_los, test_size=0.2, random_state=42)
X_train_cost, X_test_cost, y_train_cost, y_test_cost = train_test_split(X, y_cost, test_size=0.2, random_state=42)

# Model training for Length of Stay
model_los = RandomForestRegressor(n_estimators=100, random_state=42)
model_los.fit(X_train_los, y_train_los)

# Model training for Cost per Discharge
model_cost = RandomForestRegressor(n_estimators=100, random_state=42)
model_cost.fit(X_train_cost, y_train_cost)

# Predictions
pred_los = model_los.predict(X_test_los)
pred_cost = model_cost.predict(X_test_cost)

# Evaluation
rmse_los = np.sqrt(mean_squared_error(y_test_los, pred_los))
rmse_cost = np.sqrt(mean_squared_error(y_test_cost, pred_cost))

r2_los = r2_score(y_test_los, pred_los)
r2_cost = r2_score(y_test_cost, pred_cost)

# Output evaluation results
print(f"Length of Stay - RMSE: {rmse_los:.2f}, R²: {r2_los:.2f}")
print(f"Cost per Discharge - RMSE: {rmse_cost:.2f}, R²: {r2_cost:.2f}")

Length of Stay - RMSE: 0.28, R²: 0.42
Cost per Discharge - RMSE: 0.82, R²: 0.35


Length of Stay model:

RMSE: 0.28 (lower is better)
R²: 0.42 (moderate explanatory power)
Cost per Discharge model:

RMSE: 0.82
R²: 0.35 (somewhat lower predictive power but still acceptable)